# Recently somoene ask me to help her list some CEOs and comapnies in Cameroon for a project she is working on.
# i started but it was time consuming so i decided to automate the process by scrapping it since i just realize that i can get those info on the patromnal website which is https://www.legicam.cm/index.php/p/membres.

In [52]:
# importing modules for our work
#importing numpy
import numpy as np
# importing pandas
import pandas as pd
# importing request
import requests
import os, os.path
# importing beautifull soup
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
#importing the time module
import time
import datetime


In [57]:

# url that will be scrapped
url = 'https://www.legicam.cm/index.php/p/membres'
#pages_to_crawl = 20
#number of pages that will be scrapped
# here we are scrapping 40 pages which represent the number of member pages
pages_to_crawl = 40
print(url)
r= requests.get(url)
print(r)
#soup = BeautifulSoup(r.text, 'html.parser')
''' the for look bellow help get the link of each item and we can open each to investigate more in porder to get insight
from the data store in it'''
#links = soup.find_all(class_="link")
# getting all the links and putting them in an array


# scarping the pages
# here we define a for loop that will loop throught all the pages and save the link into an array
#delete the old file and create a new one
if os.path.exists('links.txt'):
    os.remove("links.txt")
f=open("links.txt","a+")
for p in range(1, pages_to_crawl+1):
    print('Scraping page:', p)
    r = requests.get(url, params={'pg' : p})
    soup = BeautifulSoup(r.text, 'html.parser')
    for linkToextract in soup.find_all('h4', attrs={'class': 'media-heading'}):
        linkOk = linkToextract.a
        link1 = linkOk.get('href')
        #print(link1)
        f.write('https://www.legicam.cm'+link1+'\n')
    #for found  in soup.h4.find_all('a', attrs={'href': re.compile("^https://")}):
        #print('in the for look')
        #link1=found.get('href')
        #print(link1)
        #f.write(link1+'\n')
f.close()
    # we pause our code for a second so that we should not be flaged as spam
   # time.sleep(1)
#     for found  in soup.findAll('a', attrs={'class':'link', 'href': re.compile("^https://")}):
#         link1=found.get('href')
#         with open ('links', 'w+') as link:
#             link.write(link1+'\n')
#print (links)


https://www.legicam.cm/index.php/p/membres
<Response [200]>
Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25
Scraping page: 26
Scraping page: 27
Scraping page: 28
Scraping page: 29
Scraping page: 30
Scraping page: 31
Scraping page: 32
Scraping page: 33
Scraping page: 34
Scraping page: 35
Scraping page: 36
Scraping page: 37
Scraping page: 38
Scraping page: 39
Scraping page: 40


In [61]:
# difining a dictionarry of data that need to be scrapped
data ={'COMPANY':[],
      'CEO':[],
      'TOWN':[],
      'HQ': [],
      'Field': [],
      'email': [],
      'phone': []
      }



#next after saving the link to each item, we open each of the and perform the data extraction on them

with open('links.txt') as f:
    for line in f:
        # open the link using the get method
        item_link = requests.get(line.strip())
        item_link_soup =BeautifulSoup(item_link.text, 'html.parser')
        if item_link_soup.find('div', {'class':'title'}).find('h1') is not None: 
            #getting the company name after inpection of the link we realize that we could get it from the div tag
            companyName = item_link_soup.find('div', {'class':'title'}).find('h1').get_text(strip=True)
            print(companyName)
            data['COMPANY'].append(companyName)
            allData = item_link_soup.find('div', {'class':'media-body'}).find_all('p')
            #print(allData)
            for p in allData:
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Activités'):
                    field = p.get_text(strip=True).replace("Activités:", "")
                    #print(field)
                    data['Field'].append(field)
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Ville'):
                    town = p.get_text(strip=True).replace("Ville:", "")
                    #print(town)
                    data['TOWN'].append(town)
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Localisation'):
                    hq = p.get_text(strip=True).replace("Localisation:", "")
                    #print(hq)
                    data['HQ'].append(hq)
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Téléphone'):
                    phone = p.get_text(strip=True).replace("Téléphone:", "")
                    #print(phone)
                    data['phone'].append(phone)
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Email'):
                    email = p.get_text(strip=True).replace("Email:", "")
                    #print(email)
                    data['email'].append(email)
                if (p.find('strong') is not None and p.find('strong').get_text(strip=True) =='Dirigeant(s)'):
                    ceo = p.get_text(strip=True).replace("Dirigeant(s):", "")
                    #print(ceo)
                    data['CEO'].append(ceo)
        # at the end the scrapper wait for 0.1 second before startint with the next link
        # note that we are waiting because we don't want to send too much request to the server in a short time because if we do so some website might block our ip
        time.sleep(0.100)
                    
#converting the dictionnary to a dataframe
frame = pd.concat([pd.Series(v, name=k) for k, v in data.items()], axis=1)
fileName = 'cameroonCompaniesInfoFromGicamWebsite_scraped_on_' + datetime.datetime.now().strftime("%c").replace(' ', '_')+('.csv')
#print(frame)
#fileNameExcel = fileName = 'cameroonCompaniesInfoFromGicamWebsite_scraped_on_' + datetime.datetime.now().strftime("%c").replace(' ', '_')+('.xlsx')
#exprot_xlsx=frame.to_excel (fileNameExcel, index = None, header=True)
# here we export the data to csv file
export_csv = frame.to_csv (fileName, index = False)
# reading the saved file

BOURSE DES VALEURS MOBILIÈRES DE L’AFRIQUE CENTRALE (BVMAC)
3T CAMEROUN
ABB (ASEA BROWN BOVERI)
ABIOGeT
AC2P (ASSOCIATION CAMEROUNAISE DES PROFESSIONNELS DE LA PLASTURGIE )
ACAM VIE S.A
ACE (African Consulting Entreprise)
ACTIVA ASSURANCES
ADDAX PETROLEUM CAMEROON Ltd
ADIC (African Distilling Company SA)
AFRICA BUSINESS CENTER
AFRICA PREMIUM CONSULTING
AFRICA WORKS
AFRILAND FIRST BANK
AFRIMAR S.A.
AFRIQUE AUDIT CONSEIL
AFRO PETRO & BUSINESS SOLUTIONS
AG PARTNERS CAMEROUN PUBLICIS AFRICA GROUP
AGGREKO CAMEROUN Ltd
AGPC (Association pharmaceutique des grossistes du Cameroun)
AGROCHEM
AIGLE VOYAGES
AJC GRAPHIC
AKWA PALACE
ALIOS FINANCES CAMEROUN
ALLIANZ CAMEROUN
ALM CONSULTANTS
ALPICAM (Alpi Pietro et Fils Cameroun Sarl)
ALUCAM (Aluminium du Cameroun)
AMD Sarl (Aciers et Métaux Divers)
APAVE CAMEROUN
APECCAM (Association Professionnelle des Etablissements de Crédit du Cameroun)
APM SA (Agence de Prestations Maritimes)
AREA ASSURANCES
ASAC (ASSOCIATION DES SOCIETES D'ASSURANCES DU CAMEROUN

SDCRA
SECA ( Services Camerounais d'Assainissement )
SECEL
SECURITY SERVICES GROUP
SEMAGRI Sarl
SEREMAC Sarl
SFID S.A.
SGMC (Société les Grands Moulins du Cameroun)
SGS CAMEROUN S.A.
SHOOTING STAR
SIC (Société Immobilière du Cameroun)
SIC CACAOS
SIKA CAMEROON
SIMTECH-3D  SARL
SIPP (SOCIETE INDUSTRIELLE DES PRODUITS PHARMACEUTIQUES)
SIVO CENTRAL AFRICA
SMPC (Société Moderne du Pneumatique Camerounais)
SMS CAMEROUN
SMT
SNAVTC (Syndicat Nationale des Agences de Voyage et de Tourisme du Cameroun)
SNI (Société Nationale des Investissements)
SNTS (Société Nationale de Transit et de Services)
SOACAM (SOCIETE ALIMENTAIRE DU CAMEROUN)
SOCAPALM (Société Camerounaise des Palmeraies)
SOCARTO (Sté Camerounaise de Cartonnages et de Fournitures de matériel  scolaire & Bureau)
SOCATAM
SOCATUR S.A.
SOCIETE  QUIFEUROU
SOCIETE DES PARCS A BOIS DU CAMEROUN
SOCIETE GENERALE CAMEROUN (SGC)
Société Nationale de Transit et de Services (SNTS)
Société TCHIN-PAS SARL
SOCIETE TIGUI SA
SOCIPEC SARL
SOCOMAR SA
SODE

In [62]:
# We convert the csv file to an excel file
df = pd.read_csv('cameroonCompaniesInfoFromGicamWebsite_scraped_on_Sun_Oct_18_083659_2020.csv')
fileNameExcel = fileName = 'cameroonCompaniesInfoFromGicamWebsite_scraped_on_' + datetime.datetime.now().strftime("%c").replace(' ', '_')+('.xlsx')
df.to_excel(fileNameExcel, index = None, header=True)